In [1]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1862)

In [2]:
df = pd.read_excel("KHV_2015.xlsx", sheet_name = "KHV_2015", skiprows= 4)

In [3]:
df.head()

,Land,Kreis,Gemeinde,Name1,Name2,Name3,Name4,Straße/Hausnr,PLZ,Ort,Telefonvorwahl/-nummer,Telefaxvorwahl/-nummer,E-Mail Adresse,Internetadresse,Traeger,T_Name1,T_Name2,T_Name3,T_Name4,Art,Betten_Ins,Augen,Chirurgie,CH_Gefäßchir,CH_Thorax,CH_Unfall,CH_Viszeral,Frauen/Geburt,F-Frau,F_Geburt,HNO,Haut_Geschlecht,Herzchir,Herz_Thorax,Innere,I_Angio,I_Endo,I_Gastro,I_Haema,I_Kardio,I_Nephro,I_Pneumo,I_Rheuma,Geriatrie,Kinderchir,Kinderhk,K_Kardiologie,K_Neonatologie,Kinderpsychiat,Mund_Kiefer,Neurochir,Neurologie,Nuklear,Ortho,O_Rheuma,Plast_Chir,Psychiat,Psychiat_Sucht,Psychotherapeut,Strahlen,Urologie,Sonst,TN_Innere,TN_I_Geriatrie,TN_Kinderpsychiat,TN_Psychiat,TN_Psychotherapeut,TN_Sonst,TN_SUM,Änd_Art,Änd_Detail
0,1,1,0,Diakonissenkrankenhaus,NaN,NaN,NaN,Knuthstraße 1,24939,Flensburg,0461 8121-100,0461 8121-107,servicecenter@diako.de,www.diako.de,4,Ev.-luth. Diakonissenanstalt,zu Flensburg,NaN,NaN,2,531,1,101,20,-,81,-,66,-,-,-,-,-,-,107,-,-,-,-,-,-,-,-,-,-,33,-,-,-,-,44,43,-,-,-,8,79,-,8,-,38,3,-,-,25,48,15,-,88,NaN,NaN
1,1,1,0,Malteser Krankenhaus,St. Franziskus Hospital,NaN,NaN,Waldstraße 17,24939,Flensburg,0461 816-0,0461 816-2012,krankenhaus.flensburg@malteser.org,www.malteser-franziskus.de,4,Malteser Norddeutschland gGmbH,NaN,NaN,NaN,2,318,-,63,-,-,-,-,-,-,-,20,-,-,-,127,-,-,-,-,-,-,-,-,102,-,-,-,-,-,3,-,-,-,-,-,-,-,-,-,-,-,3,-,37,-,-,-,-,37,NaN,NaN
2,1,2,0,Augenklinik Bellevue,NaN,NaN,NaN,Lindenallee 21,24105,Kiel,0431 30108-0,0431 30108-40,uthoff@augenklinik-bellevue.de,www.augenklinik-bellevue.de,5,Prof. Dr. med. D. Uthoff,NaN,NaN,NaN,2,35,35,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN
3,1,2,0,Helios Klinik Kiel,NaN,NaN,NaN,Hamburger Chaussee 77,24113,Kiel,0431 7206-300,0431 7206-315,NaN,www.helios-kliniken.de/klinik/kiel,5,Helios Klinik Kiel GmbH,NaN,NaN,NaN,2,50,-,14,-,-,-,-,-,-,-,1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,5,-,-,22,-,-,-,-,-,-,7,-,-,-,-,-,-,-,-,NaN,NaN
4,1,2,0,Klinik Flechsig,NaN,NaN,NaN,Chemnitzstr. 33,24116,Kiel,0431 3397100,0431 66940959,info@klinik-flechsig.de,www.klinik-flechsig.de,5,Klinik Flechsig GmbH & Co. KG,NaN,NaN,NaN,2,30,-,-,-,-,-,-,-,-,-,30,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN


In [4]:
df = df[['Land', 'Kreis', 'Gemeinde', 'Name1', 'Traeger', 'Straße/Hausnr', 'PLZ', 'Ort']]

In [5]:
df.sample(30)

,Land,Kreis,Gemeinde,Name1,Traeger,Straße/Hausnr,PLZ,Ort
1080,8,326,74,Schwarzwald-Baar-Klinikum der,3,Klinikstr. 11,78050,Villingen-Schwenningen
985,8,221,0,Klinik für Prostata-Therapie,5,Bergheimer Str. 56 a,69115,Heidelberg
1464,9,761,0,Josefinum Frauenklinik,4,Kapellenstr. 30,86154,Augsburg
1496,9,776,125,Panorama Fachklinik,5,Kurstr. 18-22,88175,Scheidegg
737,6,440,4,Capio,5,Am Schlag 9,63654,Büdingen
86,2,0,0,Asklepios Klinik Eimsbüttel,5,Eißendorfer Pferdeweg 52,21075,Hamburg
1623,12,60,20,Brandenburg Klinik,5,Brandenburgallee 1,16321,Bernau Waldsiedlung
1262,9,187,118,Medical Park,5,Rasthausstr. 25,83233,Bernau a. Chiemsee
787,6,634,9,Asklepios,5,Krankenhausstr. 27,34613,Schwalmstadt
796,6,635,13,Vitos,3,Landgraf-Philipp-Platz 3,35114,Haina


## Limit dataset to NRW

In [6]:
# df = df.loc[lambda df: df['Land'] == 5]
df = df[df.Land == 5]

In [7]:
df.sample(30)

,Land,Kreis,Gemeinde,Name1,Traeger,Straße/Hausnr,PLZ,Ort
359,5,114,0,Malteser Krankenhaus,4,Kurfürstenstr. 69,47829,Krefeld
617,5,916,0,St. Marien-Hospital Eickel,4,Marienstr. 2,44651,Herne
323,5,111,0,Fliedner Klinik,4,Martin-Luther-Platz 26,40212,Düsseldorf
322,5,111,0,Ev. Krankenhaus,4,Kirchfeldstr. 40,40217,Düsseldorf
605,5,914,0,Ev. Krankenhaus Elsey,4,Iserlohner Str. 43,58119,Hagen
494,5,382,8,Rhein-Klinik,4,Luisenstr. 3,53604,Bad Honnef
462,5,358,8,LVR-Klinik Düren,1,Meckerstr. 15,52353,Düren
331,5,111,0,Sana Kliniken Düsseldorf,5,Gräulinger Str. 120,40625,Düsseldorf
474,5,362,40,Dreifaltigkeits-Krankenhaus,4,Bonner Str. 84,50389,Wesseling
395,5,158,28,St. Marien-Krankenhaus,4,Werdener Str. 3,40878,Ratingen


## Limit dataset to one city to test API

In [13]:
# df = df.loc[lambda df: df['Ort'] == "Essen"]
df = df[df.Ort == 'Essen']

In [14]:
df.sample(10)

,Land,Kreis,Gemeinde,Name1,Traeger,Straße/Hausnr,PLZ,Ort,address
348,5,113,0,Kath. Klinikum Essen,4,Hospitalstr. 24,45329,Essen,"Hospitalstr. 24, 45329, Essen, Germany"
349,5,113,0,Kath. Klinikum Essen,4,Hülsmannstr. 17,45355,Essen,"Hülsmannstr. 17, 45355, Essen, Germany"
345,5,113,0,Elisabeth-Krankenhaus,4,Klara-Kopp-Weg 1,45138,Essen,"Klara-Kopp-Weg 1, 45138, Essen, Germany"
354,5,113,0,Ruhrlandklinik,3,Tüschener Weg 40,45239,Essen,"Tüschener Weg 40, 45239, Essen, Germany"
343,5,113,0,Alfried Krupp Krankenhaus Steele,4,Hellweg 100,45276,Essen,"Hellweg 100, 45276, Essen, Germany"
344,5,113,0,Alfried-Krupp-Krankenhaus,4,Alfried-Krupp-Str. 21,45117,Essen,"Alfried-Krupp-Str. 21, 45117, Essen, Germany"
352,5,113,0,Kliniken Essen-Süd,4,Propsteistr. 2,45239,Essen,"Propsteistr. 2, 45239, Essen, Germany"
350,5,113,0,Kliniken Essen-Mitte,4,Henricistr. 92,45136,Essen,"Henricistr. 92, 45136, Essen, Germany"
347,5,113,0,Kath. Kliniken Ruhrhalbinsel,4,Heidbergweg 22-24,45257,Essen,"Heidbergweg 22-24, 45257, Essen, Germany"
346,5,113,0,Facharztklinik Essen,5,Girardetstr. 2-38,45131,Essen,"Girardetstr. 2-38, 45131, Essen, Germany"


## Concatenate address

In [15]:
df = df.applymap(str)

In [16]:
df['address'] = df['Straße/Hausnr'].astype(str) + ', ' + \
                df['PLZ'] + ', ' + \
                df['Ort'] + ', ' + 'Germany'

In [17]:
df

,Land,Kreis,Gemeinde,Name1,Traeger,Straße/Hausnr,PLZ,Ort,address
343,5,113,0,Alfried Krupp Krankenhaus Steele,4,Hellweg 100,45276,Essen,"Hellweg 100, 45276, Essen, Germany"
344,5,113,0,Alfried-Krupp-Krankenhaus,4,Alfried-Krupp-Str. 21,45117,Essen,"Alfried-Krupp-Str. 21, 45117, Essen, Germany"
345,5,113,0,Elisabeth-Krankenhaus,4,Klara-Kopp-Weg 1,45138,Essen,"Klara-Kopp-Weg 1, 45138, Essen, Germany"
346,5,113,0,Facharztklinik Essen,5,Girardetstr. 2-38,45131,Essen,"Girardetstr. 2-38, 45131, Essen, Germany"
347,5,113,0,Kath. Kliniken Ruhrhalbinsel,4,Heidbergweg 22-24,45257,Essen,"Heidbergweg 22-24, 45257, Essen, Germany"
348,5,113,0,Kath. Klinikum Essen,4,Hospitalstr. 24,45329,Essen,"Hospitalstr. 24, 45329, Essen, Germany"
349,5,113,0,Kath. Klinikum Essen,4,Hülsmannstr. 17,45355,Essen,"Hülsmannstr. 17, 45355, Essen, Germany"
350,5,113,0,Kliniken Essen-Mitte,4,Henricistr. 92,45136,Essen,"Henricistr. 92, 45136, Essen, Germany"
351,5,113,0,Kliniken Essen-Süd,4,Pattbergstr. 1-3,45239,Essen,"Pattbergstr. 1-3, 45239, Essen, Germany"
352,5,113,0,Kliniken Essen-Süd,4,Propsteistr. 2,45239,Essen,"Propsteistr. 2, 45239, Essen, Germany"


## Get location and point

In [18]:
locator = Nominatim(user_agent="myFancyGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [19]:
df.head()

,Land,Kreis,Gemeinde,Name1,Traeger,Straße/Hausnr,PLZ,Ort,address,location,point
343,5,113,0,Alfried Krupp Krankenhaus Steele,4,Hellweg 100,45276,Essen,"Hellweg 100, 45276, Essen, Germany","(Alfried Krupp Krankenhaus Steele, 100, Hellwe...","(51.44994975, 7.0919823731518346, 0.0)"
344,5,113,0,Alfried-Krupp-Krankenhaus,4,Alfried-Krupp-Str. 21,45117,Essen,"Alfried-Krupp-Str. 21, 45117, Essen, Germany","(Alfried-Krupp-Krankenhaus, 21, Alfried-Krupp-...","(51.42623705, 7.007627003459746, 0.0)"
345,5,113,0,Elisabeth-Krankenhaus,4,Klara-Kopp-Weg 1,45138,Essen,"Klara-Kopp-Weg 1, 45138, Essen, Germany","(Klara-Kopp-Weg, Huttrop, Stadtbezirk I, Essen...","(51.4456041, 7.0331828, 0.0)"
346,5,113,0,Facharztklinik Essen,5,Girardetstr. 2-38,45131,Essen,"Girardetstr. 2-38, 45131, Essen, Germany","(2-38, Girardetstraße, Witteringsfeld, Rüttens...","(51.4308573, 7.0055116, 0.0)"
347,5,113,0,Kath. Kliniken Ruhrhalbinsel,4,Heidbergweg 22-24,45257,Essen,"Heidbergweg 22-24, 45257, Essen, Germany","(St.Josef Krankenhaus, 22-24, Heidbergweg, Dil...","(51.3916617, 7.085786276566124, 0.0)"


## Split point into lattitude, longitude and attitude

In [20]:
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [21]:
df.head()

,Land,Kreis,Gemeinde,Name1,Traeger,Straße/Hausnr,PLZ,Ort,address,location,point,latitude,longitude,altitude
343,5,113,0,Alfried Krupp Krankenhaus Steele,4,Hellweg 100,45276,Essen,"Hellweg 100, 45276, Essen, Germany","(Alfried Krupp Krankenhaus Steele, 100, Hellwe...","(51.44994975, 7.0919823731518346, 0.0)",51.449950,7.091982,0.0
344,5,113,0,Alfried-Krupp-Krankenhaus,4,Alfried-Krupp-Str. 21,45117,Essen,"Alfried-Krupp-Str. 21, 45117, Essen, Germany","(Alfried-Krupp-Krankenhaus, 21, Alfried-Krupp-...","(51.42623705, 7.007627003459746, 0.0)",51.426237,7.007627,0.0
345,5,113,0,Elisabeth-Krankenhaus,4,Klara-Kopp-Weg 1,45138,Essen,"Klara-Kopp-Weg 1, 45138, Essen, Germany","(Klara-Kopp-Weg, Huttrop, Stadtbezirk I, Essen...","(51.4456041, 7.0331828, 0.0)",51.445604,7.033183,0.0
346,5,113,0,Facharztklinik Essen,5,Girardetstr. 2-38,45131,Essen,"Girardetstr. 2-38, 45131, Essen, Germany","(2-38, Girardetstraße, Witteringsfeld, Rüttens...","(51.4308573, 7.0055116, 0.0)",51.430857,7.005512,0.0
347,5,113,0,Kath. Kliniken Ruhrhalbinsel,4,Heidbergweg 22-24,45257,Essen,"Heidbergweg 22-24, 45257, Essen, Germany","(St.Josef Krankenhaus, 22-24, Heidbergweg, Dil...","(51.3916617, 7.085786276566124, 0.0)",51.391662,7.085786,0.0


## Clean up columns

In [22]:
df.columns

Index(['Land', 'Kreis', 'Gemeinde', 'Name1', 'Traeger', 'Straße/Hausnr', 'PLZ',
       'Ort', 'address', 'location', 'point', 'latitude', 'longitude',
       'altitude'],
      dtype='object')

In [23]:
df = df.drop(['Land', 'Kreis', 'Gemeinde', 'Straße/Hausnr','PLZ', 'Ort', 'address', 'location', 'point'], axis=1)

In [24]:
df.head()

,Name1,Traeger,latitude,longitude,altitude
343,Alfried Krupp Krankenhaus Steele,4,51.449950,7.091982,0.0
344,Alfried-Krupp-Krankenhaus,4,51.426237,7.007627,0.0
345,Elisabeth-Krankenhaus,4,51.445604,7.033183,0.0
346,Facharztklinik Essen,5,51.430857,7.005512,0.0
347,Kath. Kliniken Ruhrhalbinsel,4,51.391662,7.085786,0.0


In [25]:
df.latitude.isnull().sum()

0

## Make map

In [26]:
map1 = folium.Map(
    location=[51.452441, 7.019965],
    tiles='cartodbpositron',
    zoom_start=10,
)

## Add markers

In [27]:
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)

343    <folium.vector_layers.CircleMarker object at 0...
344    <folium.vector_layers.CircleMarker object at 0...
345    <folium.vector_layers.CircleMarker object at 0...
346    <folium.vector_layers.CircleMarker object at 0...
347    <folium.vector_layers.CircleMarker object at 0...
348    <folium.vector_layers.CircleMarker object at 0...
349    <folium.vector_layers.CircleMarker object at 0...
350    <folium.vector_layers.CircleMarker object at 0...
351    <folium.vector_layers.CircleMarker object at 0...
352    <folium.vector_layers.CircleMarker object at 0...
353    <folium.vector_layers.CircleMarker object at 0...
354    <folium.vector_layers.CircleMarker object at 0...
355    <folium.vector_layers.CircleMarker object at 0...
dtype: object

In [28]:
map1

## Save map as html

In [41]:
map1.save("map1.html")

## Credits and Thanks

[Abdishakur](https://towardsdatascience.com/@shakasom) and his article [Geocode with Python](https://towardsdatascience.com/geocode-with-python-161ec1e62b89)